In [4]:
# Import necessary libraries
import pandas as pd
from scipy.spatial import distance
from sentence_transformers import SentenceTransformer

In [5]:
# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
# Input data
questions = [
    "Do shareholders have a right to obtain a copy of the company's articles of association?",
    "To whom can confidential taxpayer information be disclosed by persons employed in the administration of the Inland Revenue Act?",
    "Who is authorized to examine the books and accounts of a licensed specialised bank, and what powers do they have during such an examination?",
    "When is approval from the Securities and Exchange Commission required for a public offer of securities by a listed public company or a company applying for listing?",
    "What moral rights does an author have under the Intellectual Property Act, and are these rights transmissible?",
    "What information is a trustee obligated to provide to a beneficiary regarding the trust property?",
    "Can a contract be denied legal validity or enforceability solely because it is in electronic form?",
    "To whom can a person appeal if they are aggrieved by a decision of an authorized dealer or restricted dealer regarding foreign exchange transactions?",
    "What details must be included in an amalgamation proposal for companies?",
    "What is the effect of a certificate of conformity in an insolvency proceeding?"
]

original_answers = [
    "Yes, a shareholder has a right at any time to request a company in writing for a copy of its articles. The company must comply with such a request within five working days of receipt, though it may require a fee of not more than five hundred rupees or decline if a copy has been provided to that shareholder within the previous six months.",
    "Information and documents received in an official capacity in relation to a specific taxpayer can be disclosed only to employees of the Department of Inland Revenue and the Customs Department for the purpose of carrying out their duties, the Attorney-General's Department for opinions or advice, a court in proceedings to establish tax liability or responsibility for an offence, the Land Reform Commission for official duties, the Controller of Exchange for prosecuting Exchange Control Act violations, and a Commission appointed under the Commissions of Inquiry Act investigating a person's affairs.",
    "The Director of Bank Supervision or any authorized officer may examine the books and accounts of any licensed specialised bank. During this examination, they can administer oaths or affirmations to any director, manager, secretary, employee, or auditor, and require them to furnish information and produce books, minutes, accounts, cash, securities, vouchers, other documents, and records for inspection.",
    "A listed public company or any public company that has applied to obtain a listing in an exchange shall not make a public offer of securities, either directly or through a third party, by way of a prospectus or similar document, or otherwise for the purposes of solicitation of funds from the public, unless approved by the Commission or a person authorized by the Commission.",
    "The author of a work has the rights to have their name indicated prominently on copies and in public use, to use a pseudonym and not have their name indicated, and to object to any distortion, mutilation, or other modification of their work that would be prejudicial to their honour or reputation. These rights are not transmissible during the author's lifetime, but upon their death, the right to exercise them is transmissible by testamentary disposition or operation of law.",
    "A trustee is bound to keep clear and accurate accounts of the trust property, and at all reasonable times, upon the request of the beneficiary, to furnish them with full and accurate information as to the amount and state of the trust property.",
    "No, a contract shall not be denied legal validity or enforceability on the sole ground that it is in electronic form.",
    "Any person aggrieved by such a decision may appeal against it to the Central Bank within fourteen days after the decision is communicated. The Central Bank will then provide a reasonable opportunity for the person and the dealer to be heard, and may affirm, vary, or revoke the decision.",
    "An amalgamation proposal must set out: the name and registered office of the amalgamated company; the full name and residential address of each director and the secretary of the amalgamated company; the share structure (including number of shares, and their rights, privileges, limitations, and conditions); the manner in which shares of amalgamating companies are to be converted into shares of the amalgamated company (or any other consideration if not converted); any payment to be made to a shareholder or director; details of any arrangement necessary to complete the amalgamation and for subsequent management; and the date on which the amalgamation is intended to become effective. If the proposed articles differ from the model articles, a copy must be attached.",
    "A certificate of conformity discharges the insolvent from all debts due by him when he became insolvent and from all claims and demands made provable under the insolvency. However, it does not release or discharge any person who was a partner with the insolvent at the time of insolvency, or was then jointly bound, or had made any joint contract with the insolvent, or who was a surety for him."
]

In [7]:
chatgpt_answers = [
    "Yes. Under the Companies Act, No. 7 of 2007, a shareholder has the right at any time to request a company in writing for a copy of its Articles of Association. The company must provide it within five working days of receiving the request, and may charge a fee not exceeding five hundred rupees, unless a copy has already been provided to that shareholder within the previous six months.",
    "According to the Inland Revenue Act, No. 24 of 2017, any officer or person involved in tax administration must treat taxpayer information as confidential and cannot disclose it except:\n\n    when required by a court of law;\n\n    for the purposes of administering tax laws;\n\n    to the Auditor General or other lawful authorities under written authorization of the Commissioner General;\n\n    for statistical purposes where the information does not identify the taxpayer.",
    "Under the Banking Act, No. 30 of 1988, the Director of Bank Supervision (or an authorized officer) is empowered to examine the books, accounts, and transactions of a licensed specialised bank. During such examination, they may require officers of the bank to produce any books, records, or documents and provide explanations as necessary for the inspection.",
    "Under the Securities and Exchange Commission of Sri Lanka Act, No. 19 of 2021, a public company that is already listed or applying to be listed must obtain prior approval from the SEC before making a public offer of securities to ensure compliance with disclosure and regulatory requirements.",
    "The Intellectual Property Act, No. 36 of 2003 grants an author the moral rights:\n\n    to claim authorship of the work;\n\n    to object to any distortion, mutilation, or other modification that would prejudice their honour or reputation.\n    These moral rights are personal and not transmissible, even if the economic rights are transferred.",
    "Under the Trusts Ordinance (Cap. 87), a trustee must provide beneficiaries with full and accurate information regarding the trust property, including the accounts, investments, and administration of the trust. Beneficiaries have the right to inspect trust documents and accounts upon reasonable notice.",
    "No. The Electronic Transactions Act, No. 19 of 2006 expressly states that a contract or agreement shall not be denied legal validity, enforceability, or effect merely because it is in electronic form, provided it meets the requirements of a valid contract under Sri Lankan law.",
    "Under the Foreign Exchange Act, No. 12 of 2017, an aggrieved person can appeal to the Director of the Department of Foreign Exchange, and thereafter to the Monetary Board of the Central Bank of Sri Lanka, if dissatisfied with the decision of an authorized or restricted dealer.",
    "According to the Companies Act, No. 7 of 2007, an amalgamation proposal must include:\n\n    names of the amalgamating companies and the proposed name of the amalgamated company;\n\n    share exchange terms;\n\n    details of the property and liabilities of each company;\n\n    manner of dealing with shares held in each company;\n\n    proposed Articles of the amalgamated company;\n\n    any other material terms and conditions.",
    "In an insolvency proceeding under the Companies Act, No. 7 of 2007, a certificate of conformity (issued by the Registrar of Companies upon approval of a compromise or arrangement with creditors) serves as conclusive evidence that all legal requirements regarding the arrangement have been complied with, and the arrangement is binding on all parties involved, including dissenting creditors."
]

In [8]:
gemini_answers = [
    "Yes, under Section 19(1) of the Companies Act No. 07 of 2007, a company is required to send to every member, at his request and on payment of such sum as may be prescribed, a copy of its articles.",
    "Under Section 161(2) of the Inland Revenue Act, No. 24 of 2017, confidential taxpayer information may be disclosed in the following circumstances:\n\n    To the Minister of Finance.\n\n    For the purposes of any prosecution for an offense under the Act.\n\n    For the purposes of any legal proceedings arising from the Act.\n\n    To any person authorized in writing by the Commissioner-General for the purposes of enabling that person to perform any official duties.\n\n    To the Auditor-General for the performance of his official duties.\n\n    To any officer of the Central Bank of Sri Lanka for the purposes of monetary policy formulation and analysis.\n\n    To an authorized officer of a foreign government with which Sri Lanka has an agreement for the exchange of information, to the extent permitted by that agreement.",
    "Under Section 22 of the Banking Act No. 30 of 1988, as amended, the Monetary Board of the Central Bank of Sri Lanka is authorized to examine the books and accounts of a licensed specialised bank. During such an examination, the Monetary Board or any officer of the Central Bank authorized by it has the power to:\n\n    Require any director, officer, or employee of the bank to furnish any information and to produce any books, accounts, or other documents relating to the business of the bank.\n\n    Enter any premises of the bank.\n\n    Take copies or extracts from any books, accounts, or documents.",
    "Under Section 24 of the Securities and Exchange Commission of Sri Lanka Act No. 19 of 2021, approval from the Securities and Exchange Commission (SEC) is required for a public offer of securities by a listed public company or a company applying for listing. Specifically, a prospectus or information memorandum relating to the public offer must be submitted to and approved by the Commission prior to its publication.",
    "Under Section 10 of the Intellectual Property Act No. 36 of 2003, an author has the following moral rights:\n\n    The right to claim authorship of his work, in particular, the right to have his name indicated on the copies of his work and to appear in connection with any public use of his work (right of paternity).\n\n    The right to object to any distortion, mutilation, or other modification of, or other derogatory action in relation to, his work, which would be prejudicial to his honor or reputation (right of integrity).\n\nThese moral rights are not transmissible during the author's lifetime. However, upon the author's death, these rights devolve upon his heirs or legatees.",
    "Under Section 23 of the Trusts Ordinance (Cap. 96), a trustee is bound to give to the beneficiary, upon his application, full and accurate information as to the amount and state of the trust property.",
    "No, under Section 16 of the Electronic Transactions Act, No. 19 of 2006, a contract shall not be denied legal validity or enforceability solely on the ground that it was concluded wholly or partly by means of an electronic record.",
    "Under Section 18 of the Foreign Exchange Act No. 12 of 2017, a person aggrieved by any decision, direction, or order made by an authorized dealer or a restricted dealer may appeal to the Central Bank of Sri Lanka.",
    "Under Section 236 of the Companies Act No. 07 of 2007, an amalgamation proposal must include:\n\n    The terms and conditions of the amalgamation.\n\n    The manner in which the shares of each amalgamating company are to be converted into shares of the amalgamated company.\n\n    If any shares of an amalgamating company are not to be converted into shares of the amalgamated company, the consideration that the holders of those shares are to receive instead of shares.\n\n    Any payment to be made to a shareholder of an amalgamating company instead of the issue of fractional shares in the amalgamated company.\n\n    A statement of any material interests of the directors in the amalgamation.\n\n    A statement that the amalgamated company will be a company limited by shares, unlimited company, or company limited by guarantee, as the case may be.\n\n    The articles of the amalgamated company.\n\n    The name of the amalgamated company.",
    "Under Section 228 of the Insolvency Ordinance (Cap. 86), a certificate of conformity (granted to a bankrupt who has conformed to the provisions of the Ordinance) has the effect of discharging the bankrupt from all debts, claims, and demands provable under the insolvency, except in certain specified circumstances (e.g., debts due to the Crown, debts incurred by fraud). It essentially releases the bankrupt from their outstanding obligations."
]

In [9]:
deepseek_answers = [
    "Yes, under the Companies Act No. 07 of 2007, shareholders have the right to request and obtain a copy of the company's Articles of Association (Section 32). The company must provide this upon payment of a prescribed fee.",
    "Under the Inland Revenue Act No. 24 of 2017 (Section 6), confidential taxpayer information can only be disclosed to:\n\n    Authorized officers for tax administration purposes,\n\n    A court or tribunal in legal proceedings,\n\n    Other persons with written consent from the taxpayer, or\n\n    As required under any other written law.",
    "Under the Banking Act No. 30 of 1988 (Section 38), the Central Bank of Sri Lanka (CBSL) is authorized to examine the books and accounts of licensed specialised banks. The examiners have powers to:\n\n    Access all records,\n\n    Require explanations from directors/officers, and\n\n    Take copies of documents.",
    "Under the Securities and Exchange Commission of Sri Lanka (SEC) Act No. 36 of 1987 and SEC Rules, approval is required for:\n\n    Initial Public Offerings (IPOs),\n\n    Further issues of shares (rights issues),\n\n    Debt securities offerings, and\n\n    Any public offer requiring a prospectus (Section 51 of SEC Act).",
    "Under the Intellectual Property Act No. 36 of 2003 (Sections 10-12), an author has:\n\n    Right of paternity (to be credited),\n\n    Right of integrity (to object to distortions harming reputation).\n    These rights are personal and non-transferable, but can be waived contractually.",
    "Under the Trusts Ordinance (Chapter 87), a trustee must provide:\n\n    Details of trust assets and liabilities,\n\n    Accounts of transactions,\n\n    Any other information necessary for the beneficiary to enforce their rights (Section 83).",
    "No, under the Electronic Transactions Act No. 19 of 2006 (Section 7), electronic contracts are legally valid and enforceable, provided they meet general contract law requirements.",
    "Under the Foreign Exchange Act No. 12 of 2017, an appeal can be made to the Monetary Board of the Central Bank of Sri Lanka (Section 30).",
    "Under the Companies Act No. 07 of 2007 (Section 243), an amalgamation proposal must include:\n\n    Terms of amalgamation,\n\n    Share exchange ratios,\n\n    Details of assets/liabilities,\n\n    Rights of shareholders/creditors, and\n\n    Any special arrangements.",
    "Under the Insolvency Ordinance (Chapter 82), a Certificate of Conformity issued by the court confirms that the insolvency process complies with legal requirements. It validates the liquidator’s actions and facilitates asset distribution."
]

In [10]:
rag_answers = [
    "Yes, shareholders have a right to a copy of the articles.",
    "Confidential taxpayer information can be disclosed to: (a) employees of the Department and of the Customs Department, (b) the Minister in charge of Finance, (c) the Auditor-General, (d) tax authorities of a foreign country, (e) the Attorney-General, (f) a court, (g) the Land Reform Commission, (h) the Controller of Exchange, (i) a Commission appointed under the Commissions of Inquiry Act, and (j) the Commission to Investigate Allegations of Bribery or Corruption. Also, the Commissioner-General may disclose information to the taxpayer or their authorized representative and to another person with the taxpayer’s written consent.",
    "The Director of Bank Supervision is authorized to examine the books and accounts of a licensed specialised bank. They can also authorize any officer of their department to do so. The Director of Bank Supervision can require individuals with information about the bank to furnish it, and can examine the bank's books, records, cash balances, assets, and liabilities. They can also administer oaths, require the production of documents, and, in cases of mismanagement, require an audit by an authorized auditor.",
    "A listed public company or any public company which has applied to obtain a listing in an exchange needs approval from the Commission or a person authorized by the Commission before making a public offer of securities.",
    "An author has the right to have their name indicated on copies of their work and in connection with any public use of their work, as far as practicable. They also have the right to use a pseudonym and not have their name indicated. Additionally, they can object to any distortion, mutilation, modification, or other derogatory action in relation to their work which would be prejudicial to their honor or reputation. These rights are not transmissible during the author's lifetime, but upon death, the right to exercise these rights can be transmitted by testamentary disposition or by operation of law.",
    "A trustee is obligated to furnish a beneficiary with full and accurate information as to the amount and state of the trust property.",
    "No, a contract cannot be denied legal validity or enforceability solely because it is in electronic form.",
    "A person can appeal to the Central Bank if they are aggrieved by a decision regarding a current transaction, or to the Board of Inquiry if they are not an authorized dealer or restricted dealer and are aggrieved by a penalty.",
    "The amalgamation proposal must include: the name of the amalgamated company, the registered office of the amalgamated company, the full name and residential address of each of the directors of the amalgamated company, the full name and address of the secretary of the amalgamated company, the share structure described in paragraph (g), details of any arrangement necessary to complete the amalgamation and to provide for the subsequent management and operation of the amalgamated company, and the date on which the amalgamation is intended to become effective.",
    "The certificate of conformity discharges the insolvent from all debts due by him when he became insolvent, and from all claims and demands made provable under the insolvency. However, it does not release any partner, anyone jointly bound, anyone who made a joint contract with the insolvent, or any surety for him."
]

In [11]:
mas_answers = [
    """Yes, shareholders have a right to obtain a copy of the company's articles of association.

Referred Acts and Laws:

    Companies Act No. 07 of 2007, Section 18: "Right of shareholders to a copy of the articles".
""",
""" Persons employed in the administration of the Inland Revenue Act are generally required to keep taxpayer information secret and confidential. However, such information may be disclosed to the following individuals or entities:

    Employees of the Department of Inland Revenue and the Customs Department, for the purpose of carrying out their duties.
    The Minister in charge of Finance, for the purpose of supervising the Department.
    The Auditor-General or any authorized person by the Auditor-General, when necessary for official duties.
    Tax authorities of a foreign country, in accordance with an international agreement.
    The Attorney-General, for criminal or civil proceedings instituted by or against the State, or when their opinion or advice has been sought by the Department of Inland Revenue.
    A court, in proceedings to establish a taxpayer's tax liability or responsibility for an offence.
    The Land Reform Commission, when necessary for official duties.
    The Controller of Exchange, for prosecuting violations of the Exchange Control Act.
    A Commission appointed under the Commissions of Inquiry Act, for investigations into the affairs of any person, their spouse, or child.
    The Commission to Investigate Allegations of Bribery or Corruption.
    The taxpayer themselves or their authorized representative, after reasonable assurance of authenticity.
    Another person, with the taxpayer's written consent.

The obligation of secrecy continues even if the person ceases to be appointed under or employed in carrying out the provisions of the Act.

Referred Acts and Laws:

    Inland Revenue Act No. 24 of 2017, Section 100 (1): "Except as provided for in subsection (3), (4), (5) or (7) every person having a duty under this Act or being employed in the administration of this Act, shall regard as secret and confidential all information and documents the person has received in an official capacity in relation to a specific taxpayer, and may disclose that information only to the following persons:-"
    Inland Revenue Act No. 24 of 2017, Section 100 (1) (a): "the employees of the Department and of the Customs Department in the course, and for the purpose, of carrying out their duties;"
    Inland Revenue Act No. 24 of 2017, Section 100 (1) (b): "the Minister in charge of the subject of Finance in the course, and for the purpose, of carrying out supervision of the Department;"
    Inland Revenue Act No. 24 of 2017, Section 100 (1) (c): "the Auditor-General or any person authorised by the Auditor-General, only when such disclosure is necessary for the purposes of official duties;"
    Inland Revenue Act No. 24 of 2017, Section 100 (1) (d): "tax authorities of a foreign country, in accordance with an international agreement entered into with a specific authority;"
    Inland Revenue Act No. 24 of 2017, Section 100 (1) (e): "the Attorney-General for the purpose any criminal proceedings or civil proceedings where actions are instituted by the State or actions filed against the State or where the opinion or advice of the Attorney- General has been sought in writing by the Department of Inland Revenue;"
    Inland Revenue Act No. 24 of 2017, Section 100 (1) (f): "a court, in a proceeding to establish a taxpayer’s tax liability or responsibility for an offence;"
    Inland Revenue Act No. 24 of 2017, Section 100 (1) (g): "the Land Reform Commission, only when such disclosure is necessary for the purposes of official duties;"
    Inland Revenue Act No. 24 of 2017, Section 100 (1) (h): "the Controller of Exchange for the purpose of prosecuting violations of the Exchange Control Act;"
    Inland Revenue Act No. 24 of 2017, Section 100 (1) (i): "a Commission appointed under the Commissions of Inquiry Act, in an investigation into the affairs of any person or any person’s spouse or child;"
    Inland Revenue Act No. 24 of 2017, Section 100 (1) (j): "the Commission to Investigate Allegations of Bribery or Corruption established under the Commission to"
    Inland Revenue Act No. 24 of 2017, Section 100 (4): "The Commissioner-General may disclose information concerning a taxpayer’s affairs to the taxpayer or the taxpayer’s authorised representative only after obtaining reasonable assurance of the authenticity of the claim."
    Inland Revenue Act No. 24 of 2017, Section 100 (5): "Information concerning a taxpayer may be disclosed to another person with the taxpayer’s written consent."
""", 
""" The Director of Bank Supervision is authorized to examine the books and accounts of a licensed specialised bank. This authority also extends to any officer of the Director's department or any auditor authorized by the Director.

During such an examination, the Director of Bank Supervision or any authorized officer has the following powers:

    To examine the books, accounts, and records of the licensed specialised bank, including its cash balances, assets, and liabilities, whenever deemed necessary.
    To administer oaths or affirmations to any director, manager, secretary, employee, or auditor of the bank.
    To require any director, manager, secretary, employee, or auditor to provide all information related to the bank's affairs and to produce any relevant books, records, or documents in their possession or custody.
    In cases of suspected mismanagement, to compel any director, manager, or secretary of the bank to submit the bank's accounts for audit by an auditor authorized by the Director of Bank Supervision. The bank can also be required to provide information, produce documents, and pay fees to such an auditor.
    To issue a notice in writing requiring any person believed to have information concerning the licensed specialised bank to furnish such information or to appear in person for examination, particularly when it is necessary to determine the bank's true financial condition or if it is operating in a manner detrimental to its current or future depositors.

The licensed specialised bank, its directors, managers, secretaries, employees, and auditors are legally obligated to comply with these requirements. The Director of Bank Supervision is required to furnish a report on the examination to the Governor of the Central Bank upon completion and may also recover the costs of the examination from the licensed specialised bank.

Referred Acts and Laws:

    Banking Act No. 30 of 1988, Section 76L (1): "The Director of Bank Supervision may at any time, examine or authorise any officer of his department to examine, the books and accounts of any licensed specialised bank."
    Banking Act No. 30 of 1988, Section 76L (2): "The report on any such examination shall be furnished to the Governor of the Central Bank by the Director of Bank Supervision as soon as such examination is completed."
    Banking Act No. 30 of 1988, Section 76L (2): "The Director of Bank Supervision may recover the costs of such examination from the licensed specialised bank."
    Banking Act No. 30 of 1988, Section 76L (3)(a): "It shall be lawful for the Director of Bank Supervision or any officer authorised by him— to administer, in accordance with the Oaths Ordinance , oaths or affirmations to any director, manager, secretary, employee or auditor of any licensed specialised bank;"
    Banking Act No. 30 of 1988, Section 76L (3)(b): "It shall be lawful for the Director of Bank Supervision or any officer authorised by him— to require any director, manager, secretary, employee or auditor of any licensed specialised bank to furnish all such information relating to the affairs of the bank, as the Director of Bank Supervision or any officer authorized by him or to produce for inspection any books, records or document relating to the affairs of the bank, in his possession or custody, which is likely to contain any such information;"
    Banking Act No. 30 of 1988, Section 76L (3)(d): "In any case where there is evidence of mismanagement by a licensed specialised bank, to require any director, manager or secretary of such bank to submit the accounts of the bank for audit by an auditor authorised by the Director of Bank Supervision and to require the bank to furnish such information, or produce such books, records or documents and to pay such fees, as may be specified or authorised by the Director of Bank Supervision to such auditor."
    Banking Act No. 30 of 1988, Section 76L (4): "It shall be the duty of every director, manager, secretary, employee or any auditor of any licensed specialised bank to comply with any requirement imposed on him."
    Banking Act No. 30 of 1988, Section 76L (5): "The Director of Bank Supervision may, where he considers it necessary to ascertain the true condition of the affairs of a licensed specialised bank and to ascertain whether such bank is carrying on business in a manner detrimental to its present or future depositors, by notice in writing require any person whom he considers to have information relating to the licensed specialised bank, to furnish such information to him or to any officer authorized by him or to any auditor authorised by him under paragraph (d) of subsection (3) or to attend in person before him or any officer authorised by him at such place and at such date and time as may be specified in such notice, so that the person may be examined on such matters as may be specified in such notice."
    Banking Act No. 30 of 1988, Section 76L (6): "For the purpose of ascertaining the true condition of the affairs of the licensed specialised bank, the Director of Bank Supervision may if he thinks it necessary for the purpose of an examination under subsection (5), also examine and records of that licensed specialised bank including its cash balances, assets and liabilities whenever requested to do so by the Director of Bank Supervision."
""",
""" Approval from the Securities and Exchange Commission (SEC) is required for a public offer of securities by a listed public company or a public company applying for listing. Specifically, such companies cannot make a public offer of securities, whether directly or through a third party, via a prospectus or similar document, or for soliciting funds from the public, without the approval of the Commission or a person authorized by the Commission.

Furthermore, a listed public company must obtain approval from the Commission or an authorized person for the following specific transactions:

    Any new issue or offer for sale of securities to the public, regardless of whether it is a public offer.
    Private placement of securities.
    Rights issues of securities.
    Bonus issues of securities.
    Schemes of arrangements, schemes of reconstruction, takeover schemes, share option schemes, and the acquisition of assets through the issuance of securities.

The Commission may also require approval for certain types of public offers made by unlisted companies, taking into consideration factors such as the volume and class of securities, the number and type of investors, the nature of the issuer, or the nature of the securities market.

Referred Acts and Laws:

    Securities and Exchange Commission of Sri Lanka Act No. 19 of 2021, Section 80: "A listed public company or any public company which has applied to obtain a listing in an exchange shall not make a public offer of securities either directly or through a third party by way of a prospectus or a similar document or otherwise for the purposes of solicitation of funds from the public unless approved by the Commission or a person authorised by the Commission: Provided however, the Commission having taken into consideration the volume of securities, class of securities, the number and type of investors, the nature of the issuer or the nature of the securities market may by rules made under this Act require that the approval of the Commission be obtained prior to certain types of public offers of unlisted companies."
    Securities and Exchange Commission of Sri Lanka Act No. 19 of 2021, Section 81 (1): "A listed public company shall obtain the approval of the Commission or any person authorised by the Commission to grant approval in respect of– (a) any new issue or offer for sale of securities to the public, whether such issues or offers for sale are by way of a public offer or otherwise; (b) private placement of securities; (c) rights issues of securities; (d) bonus issues of securities; or (e) schemes of arrangements, schemes of reconstruction, take over schemes, share option schemes and acquisition of assets by way of issues of securities."
""",
""" Under the Intellectual Property Act, an author has several moral rights that are independent of their economic rights, even if they no longer own those economic rights. These moral rights include:

    The right to have their name prominently indicated on copies of their work and in connection with any public use of it, as far as practicable.
    The right to use a pseudonym and to choose not to have their name indicated on copies or in connection with any public use of their work.
    The right to object to any distortion, mutilation, or other modification of their work, or any other derogatory action that would be prejudicial to their honour or reputation.

Regarding transmissibility, these moral rights are generally not transmissible during the author's lifetime. However, upon the author's death, the right to exercise these moral rights can be transmitted through a testamentary disposition (e.g., a will) or by operation of law.

Referred Acts and Laws:

    Intellectual Property Act No. 36 of 2003, Section 11(1)(a): "to have his name indicated prominently on the copies and in connection with any public use of his work, as far as practicable."
    Intellectual Property Act No. 36 of 2003, Section 11(1)(b): "the right to use a pseudonym and not have his name indicated on the copies and in connection with any public use of his work."
    Intellectual Property Act No. 36 of 2003, Section 11(1)(c): "to object to any distortion, mutilation or other modification of, or other derogatory action in relation to, his work which would be prejudicial to his honour or reputation."
    Intellectual Property Act No. 36 of 2003, Section 11(2): "No right mentioned in subsection (1) shall be transmissible during the life time of the author, however on the death of the author, the right to exercise any of those rights shall be transmissible by testamentary disposition or by operation of law."
""",

""" A trustee is obligated to provide the following information to a beneficiary regarding the trust property:

    The trustee must keep clear and accurate accounts of the trust property.
    At all reasonable times, upon the beneficiary's request, the trustee must furnish full and accurate information regarding the amount and state of the trust property.

Referred Acts and Laws

    Trust Ordinance Cap. 96, Section 19: "A trustee is bound—(a) to keep clear and accurate accounts o f the trust property, and (b) at all reasonable times, at the request of the beneficiary, to furnish him with full and accurate information as to the amount and state of the trust property."
""",

""" No, a contract cannot be denied legal validity or enforceability solely because it is in electronic form. This principle also extends to situations where an electronic communication is used in the formation of a contract, meaning the contract's validity or enforceability cannot be denied solely on the ground that an electronic record was used for that purpose.

Referred Acts and Laws:

    Electronic Transactions Act No. 19 of 2006, Section 11: "A contract shall not be denied legal validity or enforceability on the sole ground that it is in electronic form."
    Electronic Transactions Act No. 19 of 2006, Section 17(b): "if an electronic communication is used in the formation of a contract, the contract shall not be denied validity or enforceability solely on the ground that an electronic record had been used for such purpose ;"
""",
""" If you are aggrieved by a decision of an authorized dealer or restricted dealer regarding foreign exchange transactions, the appeal process depends on the nature of the decision:

    For decisions related to current or capital foreign exchange transactions (e.g., refusal to deal): You may appeal to the Central Bank within fourteen days after the decision is communicated to you. The Central Bank will then provide you and the dealer a reasonable opportunity to be heard, and may affirm, vary, or revoke the decision.
    For authorized or restricted dealers aggrieved by specific regulatory actions: If an authorized or restricted dealer is aggrieved by a revocation of authorization or permit, a requirement to pay a determined sum, or any limitation imposed, they may appeal to the Board of Inquiry within thirty days after the decision or determination is communicated to them.
    For persons (not being authorized or restricted dealers) aggrieved by a penalty imposed: If a penalty is imposed on a person who is not an authorized or restricted dealer, they may appeal against such requirement to pay the penalty to the Board of Inquiry within thirty days after it is communicated to them.

Referred Acts and Laws:

    Foreign Exchange Act No. 12 of 2017, Section 6(6): "Any person aggrieved by the decision of an authorized dealer or a restricted dealer under subsection (4) may, within fourteen days after such decision is communicated to such person, appeal against such decision to the Central Bank and the Central Bank shall, after giving such person and the dealer a reasonable opportunity of being heard, affirm, vary or revoke such decision."
    Foreign Exchange Act No. 12 of 2017, Section 7(9): "Any person aggrieved by the decision of an an authorized dealer or a restricted dealer under subsection (7), may within fourteen days after such decision is communicated to such person, appeal against such decision to the Central Bank, and the Central Bank shall, after giving such person and the dealer a reasonable opportunity of being heard, affirm, vary or revoke such decision."
    Foreign Exchange Act No. 12 of 2017, Section 14(7)(a): "Where any authorized dealer or restricted dealer (i) aggrieved by a revocation of the authorization or permit under subsection (3); (ii) aggrieved by the requirement to pay the sum determined under paragraph (a) of subsection (4); or (iii) aggrieved by any limitation imposed under paragraph (b) of subsection (4), as the case may be, he may within thirty days after such decision or determination is communicated to such dealer, appeal against such revocation or determination to the Board of Inquiry which shall determine such appeal in accordance with the procedure set out in section 13."
    Foreign Exchange Act No. 12 of 2017, Section 15(1)(b): "Where any person not being an authorized dealer or a restricted dealer, aggrieved by a penalty imposed under subsection (5), he may within thirty days after the requirement to pay such penalty is communicated to such person, appeal against such requirement to pay such penalty to the Board of Inquiry which shall determine such appeal in accordance with the procedure."
""", 
""" An amalgamation proposal for companies must include the following details:

    The name of the amalgamated company, especially if it is the same as one of the amalgamating companies.
    The registered office of the amalgamated company.
    The full name and residential address of each director of the amalgamated company.
    The full name and address of the secretary of the amalgamated company.
    Details of any arrangement necessary to complete the amalgamation and to provide for the subsequent management and operation of the amalgamated company.
    The date on which the amalgamation is intended to become effective.
    If the proposed articles of the amalgamated company are different from the model articles, a copy of the proposed articles must be attached to and form part of the amalgamation proposal.
    If shares of one amalgamating company are held by or on behalf of another amalgamating company, the proposal must include provisions for the cancellation of those shares without payment or the provision of other consideration when the amalgamation becomes effective.

Referred Acts and Laws:

    Companies Act No. 07 of 2007, Section 241: "Every company which proposes to amalgamate shall approve in accordance with the provisions of section 241 an amalgamation proposal setting out the terms of the amalgamation, and in particular — the name of the amalgamated company, if it is the same as the name of one of the amalgamating companies."
    Companies Act No. 07 of 2007, Section 241: "Every company which proposes to amalgamate shall approve in accordance with the provisions of section 241 an amalgamation proposal setting out the terms of the amalgamation, and in particular — the registered office of the amalgamated company."
    Companies Act No. 07 of 2007, Section 241: "Every company which proposes to amalgamate shall approve in accordance with the provisions of section 241 an amalgamation proposal setting out the terms of the amalgamation, and in particular — the full name and residential address of each of the directors of the amalgamated company."
    Companies Act No. 07 of 2007, Section 241: "Every company which proposes to amalgamate shall approve in accordance with the provisions of section 241 an amalgamation proposal setting out the terms of the amalgamation, and in particular — the full name and address of the secretary of the amalgamated company."
    Companies Act No. 07 of 2007, Section 241: "Every company which proposes to amalgamate shall approve in accordance with the provisions of section 241 an amalgamation proposal setting out the terms of the amalgamation, and in particular — details of any arrangement necessary to complete the amalgamation and to provide for the subsequent management and operation of the amalgamated company."
    Companies Act No. 07 of 2007, Section 241: "Every company which proposes to amalgamate shall approve in accordance with the provisions of section 241 an amalgamation proposal setting out the terms of the amalgamation, and in particular — the date on which the amalgamation is intended to become effective."
    Companies Act No. 07 of 2007, Section 241: "If the proposed articles of the amalgamated company are different from the model articles, a copy of the proposed articles shall be attached to and shall form part of the amalgamation proposal."
    Companies Act No. 07 of 2007, Section 241: "Where shares of one of the amalgamating companies are held by or on behalf of another of the amalgamating companies, the amalgamation proposal shall provide for the cancellation of those shares without payment or the provisions of other consideration when the amalgamation becomes effective."
""", 
""" A certificate of conformity in an insolvency proceeding has several key effects:

    Discharge of Debts and Claims: It discharges the insolvent person from all debts owed by them at the time of their insolvency, as well as from all claims and demands that were provable under the insolvency.
    No Release for Others: It does not, however, release or discharge any person who was a partner with the insolvent at the time of insolvency, or who was jointly bound or had made a joint contract with the insolvent, or who acted as a surety for the insolvent.
    Future Acquisitions Unaffected: The certificate or any execution based on it does not affect any estate or effects acquired by the insolvent after the certificate of conformity has been allowed.
    Protection from Arrest and Pleading: An insolvent who has obtained their certificate is free from arrest and may use the certificate as evidence in legal proceedings. They are also not liable for any promise to pay a debt that was discharged by the certificate.
    Conditions for Validity and Recall: The certificate may be void if the insolvent concealed or falsified books, concealed property, or allowed fictitious debts to be proven. It can also be recalled by the Court of Appeal within six months of its allowance under certain conditions, such as the insolvent having concealed or falsified books, concealed property, or permitted fictitious debts to be proved.

Referred Acts and Laws:

    INSOLVENTS [Cap.103], Section 126: "The certificate o f conformit y allowed unde r thi s Ordinance, subjec t to the provision s herein contained, shall discharge the insolven t fro m all debt s du e by him when he became insolvent, and fro m all claims and demand s made provabl e unde r the insolvency : Provide d always tha t n o such certificate shall release or discharge an y person wh o was a partne r wit h suc h insolven t at th e time o f his insolvency, or was the n jointl y boun d or had made an y join t contrac t wit h such insolvent, or who was a suret y for him."
    INSOLVENTS [Cap.103], Section 153: "no r shall an y suc h certificat e or executio n in an y manne r affec t any estat e or effects which shal l come to or be acquired by th e insolvent , afte r th e allowance of his certificate o f conformity ."
    INSOLVENTS [Cap.103], Section 127: "No insolvent shall be entitled to a certificat e of conformit y unde r thi s Ordinance, and any such certificate if allowed shall be void , if such insolvent shal l after an act o f insolvency , or"
    INSOLVENTS [Cap.103], Section 129: "At any time within six months after any certificat e of conformit y shall have been allowed, and subject to such orde r as to deposit o f costs a s may be made by the Cour t of Appeal , an y credito r of th e Effec t o f certificate."
    INSOLVENTS [Cap.103], Section 129: "Certificate no t grante d or void if insolven t has concealed or falsified books, &c., or concealed any property, or permitte d any fictitious debts t o be proved."
    INSOLVENTS [Cap.103], Section 129: "Insolven t no t liable upo n an y promise to pay deb t discharged by certificate ."
    INSOLVENTS [Cap.103], Section 129: "Insolvent, havin g obtaine d hi s certificate , free from arrest ; may plead his certificat e ; evidence under it."
"""
]

In [13]:
# Encode original answers once
original_vecs = [model.encode([ans])[0] for ans in original_answers]

# Helper function using scipy cosine distance
def cosine_sim(vec1, vec2):
    return 1 - distance.cosine(vec1, vec2)

answer_dict = {
    'ChatGPT': chatgpt_answers,
    'Gemini': gemini_answers,
    'DeepSeek': deepseek_answers,
    'RAG': rag_answers,
    'MAS': mas_answers
}

In [14]:
results = []
for i, orig_vec in enumerate(original_vecs):
    row = {'Question': questions[i]}
    for method, answers in answer_dict.items():
        method_vec = model.encode([answers[i]])[0]
        row[method] = cosine_sim(orig_vec, method_vec)
    results.append(row)

df = pd.DataFrame(results)
df.set_index('Question', inplace=True)
avg_scores = df.mean(axis=0).to_frame(name='Average Similarity Score')

In [15]:
# Display the results in a DataFrame
df

,ChatGPT,Gemini,DeepSeek,RAG,MAS
Question,,,,,
Do shareholders have a right to obtain a copy of the company's articles of association?,0.876248,0.601896,0.700861,0.778211,0.713495
To whom can confidential taxpayer information be disclosed by persons employed in the administration of the Inland Revenue Act?,0.666194,0.678548,0.744998,0.826704,0.859828
"Who is authorized to examine the books and accounts of a licensed specialised bank, and what powers do they have during such an examination?",0.908130,0.710222,0.640520,0.932928,0.911714
When is approval from the Securities and Exchange Commission required for a public offer of securities by a listed public company or a company applying for listing?,0.677824,0.730376,0.603117,0.806791,0.757734
"What moral rights does an author have under the Intellectual Property Act, and are these rights transmissible?",0.581807,0.719789,0.516858,0.967413,0.796848
What information is a trustee obligated to provide to a beneficiary regarding the trust property?,0.733976,0.807226,0.686241,0.878023,0.856331
Can a contract be denied legal validity or enforceability solely because it is in electronic form?,0.752751,0.815385,0.732129,0.944137,0.853495
To whom can a person appeal if they are aggrieved by a decision of an authorized dealer or restricted dealer regarding foreign exchange transactions?,0.532640,0.568592,0.470286,0.771044,0.769603
What details must be included in an amalgamation proposal for companies?,0.878454,0.871531,0.844537,0.853366,0.899193


In [16]:
avg_scores

,Average Similarity Score
ChatGPT,0.730140
Gemini,0.721976
DeepSeek,0.659826
RAG,0.872423
MAS,0.824538
